In [1]:
! python fredmd/transform.py

--- Démarrage du traitement de fredmd/fredmd.csv ---
Traitement des colonnes...

--- Nettoyage du 'Ragged Edge' (fin de fichier) ---
Suppression de 3 ligne(s) incomplète(s) à la fin du fichier (Ragged Edge).
Lignes restantes : 801

--- Rapport de validité des données ---
(Affiche uniquement les colonnes où des données manquent au-delà de la transformation mathématique)
Colonne 'PERMIT' (Code 4) : Incomplète jusqu'au 1959-12-01. (Valide dès 1960-01-01)
Colonne 'PERMITNE' (Code 4) : Incomplète jusqu'au 1959-12-01. (Valide dès 1960-01-01)
Colonne 'PERMITMW' (Code 4) : Incomplète jusqu'au 1959-12-01. (Valide dès 1960-01-01)
Colonne 'PERMITS' (Code 4) : Incomplète jusqu'au 1959-12-01. (Valide dès 1960-01-01)
Colonne 'PERMITW' (Code 4) : Incomplète jusqu'au 1959-12-01. (Valide dès 1960-01-01)
Colonne 'ACOGNO' (Code 5) : Incomplète jusqu'au 1992-02-01. (Valide dès 1992-03-01)
Colonne 'ANDENOx' (Code 5) : Incomplète jusqu'au 1968-02-01. (Valide dès 1968-03-01)
Colonne 'CP3Mx' (Code 2) : Incomp

# Comparing Models one month ahead forecast between 2013 and 2019

In [2]:
from model import *
import numpy as np

import pandas as pd
data = pd.read_csv('fredmd/nice_fredmd.csv')
data.set_index('sasdate', inplace=True)
data.index = pd.to_datetime(data.index)
data = data[data.index >= "1970"]
data = data[data.index < "2020"]
cols_with_nan = data.columns[data.isna().any()].tolist()
if len(cols_with_nan) > 0:
    print(f"--- {len(cols_with_nan)} Colonnes with NaN to deleate ---")
    for col in cols_with_nan:
        print(f"- {col}")
else:
    print("No NaN detected.")
data = data.dropna(axis=1)

from dataloader import MacroDataLoader

X_col = list(data.columns)
Y_col = ['RPI', 'CPIAUCSL', 'UNRATE', 'GS10']
cfg = {
    'date_col': 'sasdate',
    'country_col': None,
    'target_col': Y_col,
    'features': X_col
}
loader = MacroDataLoader(data, cfg)

models_dict = {
    'RNN': make_builder(RNNModel, base_config=config_rnn, weight_decay=1e-2),
    'NODE': make_builder(LatentNODEModel, base_config=config_node, weight_decay=1e-2),
    'FVAR': build_fvar,
    'ESN': build_esn_panel,
    'OLS-VAR': build_var,
    'Ridge-VAR': build_ridge_cv,
    'RF': build_rf_panel,
    'Minnesota-VAR': build_minnesota_ridge,
}

from evaluator import UniFreqPanelMacroEvaluator

evaluator = UniFreqPanelMacroEvaluator(
    dataloader=loader,
    models_builders=models_dict,
    horizon=2,
    verbose=True,
    metric='rmse'
)

backtest_dates = pd.date_range(start='2013-01', periods=12*6, freq='ME')

results = evaluator.run_backtesting(
    end_train_date_list=backtest_dates,
    num_exp_by_fit=12
)

evaluator.print_significance_tests(results, target_idx=None, horizon_idx=0)

for i in range(len(cfg['target_col'])):
    print(f"\n {cfg['target_col'][i]}")
    evaluator.print_significance_tests(results, target_idx=i, horizon_idx=0)

--- 3 Colonnes with NaN to deleate ---
- ACOGNO
- TWEXAFEGSMTHx
- UMCSENTx
[MacroDataLoader] Initialized: 1 entities, 600 obs, 123 features.

[Backtest] Model: RNN

[Backtest] Model: NODE

[Backtest] Model: FVAR

[Backtest] Model: ESN

[Backtest] Model: OLS-VAR

[Backtest] Model: Ridge-VAR

[Backtest] Model: RF

[Backtest] Model: Minnesota-VAR

STATISTICAL SIGNIFICANCE TESTS
  Target  : All Targets (sum)
  Horizon : H1

1. FRIEDMAN TEST  (Non-parametric, multiple models)
----------------------------------------------------------------------------------------------------
Statistic : 103.1111
p-value   : 0.0000  →  ✓ Significant (α=0.05)

Mean Ranks (lower = better):
        Model  Mean Rank  Mean RMSE
Minnesota-VAR   3.625000   1.117948
          RNN   3.930556   1.129417
          ESN   3.930556   1.121476
         FVAR   4.208333   1.162142
    Ridge-VAR   4.319444   1.143179
         NODE   4.347222   1.158539
           RF   4.486111   1.168628
      OLS-VAR   7.152778   1.874022




# For covid crisis, between 2020-01 and 2021-10

In [3]:
from model import *
import numpy as np

import pandas as pd
data = pd.read_csv('fredmd/nice_fredmd.csv')
data.set_index('sasdate', inplace=True)
data.index = pd.to_datetime(data.index)
data = data[data.index >= "1970"]
data = data[data.index < "2022"]
cols_with_nan = data.columns[data.isna().any()].tolist()
if len(cols_with_nan) > 0:
    print(f"--- {len(cols_with_nan)} Colonnes with NaN to deleate ---")
    for col in cols_with_nan:
        print(f"- {col}")
else:
    print("No NaN detected.")
data = data.dropna(axis=1)

from dataloader import MacroDataLoader

X_col = list(data.columns)
Y_col = ['RPI', 'CPIAUCSL', 'UNRATE', 'GS10']
cfg = {
    'date_col': 'sasdate',
    'country_col': None,
    'target_col': Y_col,
    'features': X_col
}
loader = MacroDataLoader(data, cfg)

models_dict = {
    'RNN': make_builder(RNNModel, base_config=config_rnn, weight_decay=1e-2),
    'NODE': make_builder(LatentNODEModel, base_config=config_node, weight_decay=1e-2),
    'FVAR': build_fvar,
    'ESN': build_esn_panel,
    'OLS-VAR': build_var,
    'Ridge-VAR': build_ridge_cv,
    'RF': build_rf_panel,
    'Minnesota-VAR': build_minnesota_ridge,
    }

from evaluator import UniFreqPanelMacroEvaluator

evaluator_crisis = UniFreqPanelMacroEvaluator(
    dataloader=loader,
    models_builders=models_dict,
    horizon=2,
    verbose=True,
    metric='rmse'
)

backtest_dates = pd.date_range(start='2020-01', periods=22, freq='ME')

results_crisis = evaluator_crisis.run_backtesting(
    end_train_date_list=backtest_dates,
    num_exp_by_fit=6
)

print('\n Average')

evaluator_crisis.print_significance_tests(results_crisis, target_idx=None, horizon_idx=0)

for i in range(len(cfg['target_col'])):
    print(f"\n {cfg['target_col'][i]}")
    evaluator.print_significance_tests(results_crisis, target_idx=i, horizon_idx=0)

--- 5 Colonnes with NaN to deleate ---
- ACOGNO
- CP3Mx
- COMPAPFFx
- TWEXAFEGSMTHx
- UMCSENTx
[MacroDataLoader] Initialized: 1 entities, 624 obs, 121 features.

[Backtest] Model: RNN

[Backtest] Model: NODE

[Backtest] Model: FVAR

[Backtest] Model: ESN

[Backtest] Model: OLS-VAR

[Backtest] Model: Ridge-VAR

[Backtest] Model: RF

[Backtest] Model: Minnesota-VAR

 Average

STATISTICAL SIGNIFICANCE TESTS
  Target  : All Targets (sum)
  Horizon : H1

1. FRIEDMAN TEST  (Non-parametric, multiple models)
----------------------------------------------------------------------------------------------------
Statistic : 37.4242
p-value   : 0.0000  →  ✓ Significant (α=0.05)

Mean Ranks (lower = better):
        Model  Mean Rank  Mean RMSE
          RNN   3.136364   7.078973
         NODE   3.454545   7.214385
Minnesota-VAR   3.681818   7.236105
           RF   3.818182   6.943013
          ESN   4.681818   7.328102
    Ridge-VAR   5.227273   7.822937
         FVAR   5.272727   8.921976
      OLS